In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
import tensorflow
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tensorflow.set_random_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)

#### The Data

In [4]:
train_set = pd.read_csv('data/mimic/training_set.csv')
val_set = pd.read_csv('data/mimic/validation_set.csv')
test_set = pd.read_csv('data/mimic/test_set.csv')
train_set.shape, val_set.shape, test_set.shape

((32009, 27), (8003, 27), (10004, 27))

In [5]:
train_set.head()

,max_ret_count,mean_ret_count,min_ret_count,max_ferritin,mean_ferritin,min_ferritin,max_hemoglobin,mean_hemoglobin,min_hemoglobin,max_iron,...,min_rbc,max_segmented_neutrophils,mean_segmented_neutrophils,min_segmented_neutrophils,max_tibc,mean_tibc,min_tibc,age,gender,diagnosis
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,10.4,9.68,8.4,-1.0,...,2.57,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,86.0,1,2
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,11.3,9.52,7.8,-1.0,...,2.73,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,76.0,1,2
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,10.9,10.83,10.8,-1.0,...,3.51,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,87.0,1,0
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,11.0,11.00,11.0,-1.0,...,3.97,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,37.0,0,0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,10.2,9.48,8.8,-1.0,...,2.94,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,68.0,0,4


In [6]:
def replace_with_nans(df, old_value, new_value):
    revised_df = df.replace([old_value], new_value)
    return revised_df

In [7]:
entire_train_set = pd.concat([train_set, val_set], axis=0)
entire_train_set.shape

(40012, 27)

In [8]:
entire_train_set = replace_with_nans(entire_train_set, -1, 0)
test_set = replace_with_nans(test_set, -1, 0)
entire_train_set.head()

,max_ret_count,mean_ret_count,min_ret_count,max_ferritin,mean_ferritin,min_ferritin,max_hemoglobin,mean_hemoglobin,min_hemoglobin,max_iron,...,min_rbc,max_segmented_neutrophils,mean_segmented_neutrophils,min_segmented_neutrophils,max_tibc,mean_tibc,min_tibc,age,gender,diagnosis
0,0.0,0.0,0.0,0.0,0.0,0.0,10.4,9.68,8.4,0.0,...,2.57,0.0,0.0,0.0,0.0,0.0,0.0,86.0,1,2
1,0.0,0.0,0.0,0.0,0.0,0.0,11.3,9.52,7.8,0.0,...,2.73,0.0,0.0,0.0,0.0,0.0,0.0,76.0,1,2
2,0.0,0.0,0.0,0.0,0.0,0.0,10.9,10.83,10.8,0.0,...,3.51,0.0,0.0,0.0,0.0,0.0,0.0,87.0,1,0
3,0.0,0.0,0.0,0.0,0.0,0.0,11.0,11.00,11.0,0.0,...,3.97,0.0,0.0,0.0,0.0,0.0,0.0,37.0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,10.2,9.48,8.8,0.0,...,2.94,0.0,0.0,0.0,0.0,0.0,0.0,68.0,0,4


In [9]:
class_dict = {'Iron deficiency anemia': 0,
              'Vitamin B12/Folate deficiency anemia': 1,
              'Hemolytic anemia': 2,
              'Aplastic anemia': 3,
              'Anemia of chronic disease': 4,
              'No anemia': 5
              }

In [10]:
X_train = entire_train_set.iloc[:, 0:-1]
y_train = entire_train_set.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)

X_test = test_set.iloc[:, 0:-1]
y_test = test_set.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((40012, 26), (40012,), (10004, 26), (10004,))

In [11]:
action_list = list(class_dict.keys()) + [col  for col in entire_train_set.columns if col!='diagnosis']
len(action_list)

32

In [12]:
action_list

['Iron deficiency anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Anemia of chronic disease',
 'No anemia',
 'max_ret_count',
 'mean_ret_count',
 'min_ret_count',
 'max_ferritin',
 'mean_ferritin',
 'min_ferritin',
 'max_hemoglobin',
 'mean_hemoglobin',
 'min_hemoglobin',
 'max_iron',
 'mean_iron',
 'min_iron',
 'max_mcv',
 'mean_mcv',
 'min_mcv',
 'max_rbc',
 'mean_rbc',
 'min_rbc',
 'max_segmented_neutrophils',
 'mean_segmented_neutrophils',
 'min_segmented_neutrophils',
 'max_tibc',
 'mean_tibc',
 'min_tibc',
 'age',
 'gender']

In [13]:
X_train[:2], X_test[:2]

(array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , 10.4 ,  9.68,  8.4 ,
          0.  ,  0.  ,  0.  , 99.  , 97.  , 96.  ,  3.22,  3.01,  2.57,
          0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , 86.  ,  1.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , 11.3 ,  9.52,  7.8 ,
          0.  ,  0.  ,  0.  , 90.  , 86.74, 80.  ,  4.21,  3.35,  2.73,
          0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , 76.  ,  1.  ]]),
 array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         9.400e+00, 9.300e+00, 9.200e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         8.900e+01, 8.850e+01, 8.800e+01, 3.150e+00, 3.140e+00, 3.140e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         8.500e+01, 1.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, 1.008e+03, 1.008e+03, 1.008e+03,
         1.140e+01, 8.440e+00, 6.700e+00, 2.300e+01, 2.300e+01, 2.300e+01,
         8.800e+01, 8.234e+01, 7.800e+01, 4.560e+00, 3.360e+00, 2.600e+00,
         0.000e+00, 0

#### The Environment

In [14]:
from envs import MimicEnv

#### The Agent

In [16]:
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from stable_baselines import DQN
from stable_baselines import bench, logger

In [17]:
def stable_dqn():
    training_env = MimicEnv(X_train, y_train)
    env = bench.Monitor(training_env, logger.get_dir())
    model = DQN('MlpPolicy', training_env, verbose=1, seed=SEED, n_cpu_tf_sess=1)
    model.learn(total_timesteps=int(5e6), log_interval=10000)
    #model.learn(total_timesteps=int(1.2e5), log_interval=10000)
    #model.save('models/synthetic_stable_dqn_1.8.pkl')
    model.save('models/mimic/dqn_5e6.pkl')
    env.close()
    return model

dqn_model = stable_dqn()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
--------------------------------------
| % time spent exploring  | 89       |
| episodes                | 10000    |
| mean 100 episode reward | 2.7      |
| steps                   | 52343    |
| success rate            | 0.1      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 78       |
| episodes                | 20000    |
| mean 100 episode reward | 1.8      |
| steps                   | 109726   |
| success rate            | 0.15     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 65       |
| episodes                | 30000    |
| mean 100 episode reward | 1.6      |
| steps                   | 173719   |
| success rate            | 0.14     |
--------------------------------------


--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 280000   |
| mean 100 episode reward | -9       |
| steps                   | 3432384  |
| success rate            | 0        |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 290000   |
| mean 100 episode reward | -8.7     |
| steps                   | 3566481  |
| success rate            | 0        |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 300000   |
| mean 100 episode reward | -8.4     |
| steps                   | 3702099  |
| success rate            | 0.02     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 310000   |
| mean 100 episode reward | -9.2     |
| steps                  

#### Performance Evaluation

In [18]:
#dqn_model = DQN.load('models/synthentic_with_hb_some_nans_stable_dqn2e6.pkl')

In [19]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, auc, roc_curve

In [20]:
def multiclass(actual_class, pred_class, average = 'macro'):

    unique_class = set(actual_class)
    roc_auc_dict = {}
    for per_class in unique_class:
        other_class = [x for x in unique_class if x != per_class]
        new_actual_class = [0 if x in other_class else 1 for x in actual_class]
        new_pred_class = [0 if x in other_class else 1 for x in pred_class]
        roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
        roc_auc_dict[per_class] = roc_auc
    avg = sum(roc_auc_dict.values()) / len(roc_auc_dict)
    return avg

In [21]:
def test(ytest, ypred):
    acc = accuracy_score(ytest, ypred)
    f1 = f1_score(ytest, ypred, average ='macro', labels=np.unique(ytest))
    try:
        roc_auc = multiclass(ytest, ypred)
    except:
        roc_auc = None
    return acc, f1, roc_auc

In [22]:
def get_avg_length_reward(df):
    length = np.mean(df.episode_length)
    reward = np.mean(df.reward)
    return length, reward

In [23]:
def synthetic_dqn_eval(dqn_model):
    attempts, correct = 0,0
    test_df = pd.DataFrame()

    env = MimicEnv(X_test, y_test, random=False)
    #env = SyntheticComplexHbEnv(X_train, y_train, random=False)
    count=0

    try:
        while True:
            count+=1
            if count%5000==0:
                print(f'Count: {count}')
            obs, done = env.reset(), False
            while not done:
                action, _states = dqn_model.predict(obs, deterministic=True)
                obs, rew, done,info = env.step(action)
                #if (done==True) & (np.isfinite(info['y_pred'])):
                if done == True:
                    test_df = test_df.append(info, ignore_index=True)
                #print('....................TEST DF ....................')
                #if len(test_df) != 0:
                #    print(test_df.head())

    except StopIteration:
        print('Testing done.....')
    return test_df

test_df = synthetic_dqn_eval(dqn_model)

Count: 5000
Count: 10000
Testing done.....


In [24]:
len(X_train), len(y_train)

(40012, 40012)

In [25]:
len(X_test), len(test_df)

(10004, 10004)

In [26]:
y_pred_df = test_df[test_df['y_pred'].notna()]
success_df = y_pred_df[y_pred_df['y_pred']== y_pred_df['y_actual']]
len(success_df)

0

In [27]:
# test_df.to_csv('test_dfs/train_test_df_with_hb_some_nans_2e6.csv', index=False)
# y_pred_df.to_csv('test_dfs/train_y_pred_df_with_hb_some_nans_2e6.csv', index=False)
# success_df.to_csv('test_dfs/train_success_df_with_hb_some_nans_2e6.csv', index=False)

In [28]:
y_pred_df.y_pred.unique()

array([], dtype=float64)

In [29]:
y_pred_df.y_pred.value_counts()

Series([], Name: y_pred, dtype: int64)

In [30]:
y_pred_df.y_actual.value_counts()

Series([], Name: y_actual, dtype: int64)

In [31]:
success_rate = len(success_df)/len(test_df)*100
success_rate

0.0

In [32]:
#avg length and return 
avg_length, avg_return = get_avg_length_reward(test_df)
avg_length, avg_return

(14.0, -11.365053978408637)

In [33]:
acc, f1, roc_auc = test(y_pred_df['y_actual'], y_pred_df['y_pred'])
acc, f1, roc_auc

C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(nan, nan, None)

#### Confusion Matrix and Classification Report

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
def plot_confusion_matrix(cm, save=False, filename=False):
    cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5], columns = [0, 1, 2, 3, 4, 5])
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_df, annot=True)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Anemia')
    plt.xlabel('Predicted Anemia')
    plt.tight_layout()
    if save:
        plt.savefig(filename)
    plt.show()
    plt.close()

In [ ]:
plot_confusion_matrix(confusion_matrix(y_pred_df['y_actual'], y_pred_df['y_pred']))

In [ ]:
def cm2inch(*tupl):
    inch = 2.54
    if type(tupl[0]) == tuple:
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)

In [ ]:
def show_values(pc, fmt="%.2f", **kw):    
    pc.update_scalarmappable()
    ax = pc.axes
    for p, color, value in zip(pc.get_paths(), pc.get_facecolors(), pc.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha="center", va="center", color=color, **kw)

In [ ]:
def heatmap(AUC, title, xlabel, ylabel, xticklabels, yticklabels, figure_width=40, figure_height=20, correct_orientation=False, cmap='RdBu'):
    fig, ax = plt.subplots()    
    c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap=cmap)
    ax.set_yticks(np.arange(AUC.shape[0]) + 0.5, minor=False)
    ax.set_xticks(np.arange(AUC.shape[1]) + 0.5, minor=False)
    ax.set_xticklabels(xticklabels, minor=False)
    ax.set_yticklabels(yticklabels, minor=False)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)      

    # Remove last blank column
    plt.xlim( (0, AUC.shape[1]) )

    # Turn off all the ticks
    ax = plt.gca()    
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    # Add color bar
    plt.colorbar(c)

    # Add text in each cell 
    show_values(c)

    # Proper orientation (origin at the top left instead of bottom left)
    if correct_orientation:
        ax.invert_yaxis()
        ax.xaxis.tick_top()       

    # resize 
    fig = plt.gcf()
    fig.set_size_inches(cm2inch(figure_width, figure_height))

In [ ]:
def plot_classification_report(classification_report, save=False, filename=False, cmap='RdBu'):
    lines = classification_report.split('\n')
    class_names = list(class_dict.keys())
    plotMat = []
    support = []
    #class_names = []
    #count = 0
    for line in lines[2 : (len(lines) - 5)]:
        t = line.strip().split()
        if len(t) < 2: continue
        v = [float(x) for x in t[1: len(t) - 1]]
        support.append(int(t[-1]))
        plotMat.append(v)

    xlabel = 'Metrics'
    ylabel = 'Classes'
    xticklabels = ['Precision', 'Recall', 'F1-score']
    ytick_labels = [f'{class_names[i]}({sup})' for i, sup in enumerate(support) ]
    
    #print(len(support))
    yticklabels = ['{0} ({1})'.format(class_names[idx], sup) for idx, sup  in enumerate(support)]
    figure_width = 25
    figure_height = len(class_names) + 7
    correct_orientation = False
    heatmap(np.array(plotMat), 'classification report', xlabel, ylabel, xticklabels, yticklabels, figure_width, figure_height, correct_orientation, cmap=cmap)
    #plt.tight_layout()
    if save:
        plt.savefig(filename, bbox_inches = 'tight')
    plt.show()
    plt.close()

In [ ]:
cr = classification_report(y_pred_df['y_actual'], y_pred_df['y_pred'])
plot_classification_report(cr)